In [ ]:
import sys
sys.path.append('..')  # Add the parent directory to sys.path

In [ ]:
import torch
from src.utils import train, set_device, beam_search
from src.models import GenerativeRNN, GenerativeLSTM


In [ ]:
SEED = 265
torch.manual_seed(SEED)
DEVICE = set_device("cuda")
print(f"Using device: {DEVICE}")

In [ ]:
data_train = torch.load("../generated_data/data_train.pt")
data_val = torch.load("../generated_data/data_val.pt")
data_test = torch.load("../generated_data/data_test.pt")
mapping = torch.load("../generated_data/mapping.pt")
embedding = torch.load("../generated_data/embedding_matrix.pt")
vocab = torch.load("../generated_data/vocabulary.pt")